In [ ]:
!pip install tqdm
!pip install pyngrok
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import socket
import tqdm
import os
from pyngrok import ngrok

port = 5311
ngrok.set_auth_token("2YAVMnJuQSyzdmvIgDE5G377dya_4fNS8ZKGC3Sj69HEKD6DU")
public_url = ngrok.connect(port, "tcp")
print(public_url)

filename_out = "01-1111.mkv"
path = '/content/drive/My Drive/Colab Notebooks/' + filename_out

SERVER_HOST = "localhost"
SERVER_PORT = port

BUFFER_SIZE = 4096
SEPARATOR = "<SEPARATOR>"

s = socket.socket()
s.bind((SERVER_HOST, SERVER_PORT))
s.listen(5)
print(f"[*] Listening as {SERVER_HOST}:{SERVER_PORT}")
client_socket, address = s.accept()
print(f"[+] {address} is connected.")
received = client_socket.recv(BUFFER_SIZE).decode()
filename, filesize = received.split(SEPARATOR)
filename = os.path.basename(filename)
filesize = int(filesize)

progress = tqdm.tqdm(range(filesize), f"Receiving {filename}", unit="B", unit_scale=True, unit_divisor=1024)
with open(path, "wb") as f:
    while True:
        bytes_read = client_socket.recv(BUFFER_SIZE)

        if not bytes_read:
            break

        f.write(bytes_read)
        progress.update(len(bytes_read))

client_socket.close()
s.close()